In [2]:
from __future__ import print_function, division

#from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from tqdm import notebook
import os
import sys
import numpy as np

In [3]:
def load_data():
  #data_x_p1 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_x_Bacteria.npy")
  #data_x_p2 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_x_Normal.npy")
  data_x_p3 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_x_Virus.npy")
  #data_x_p4 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_x_Covid.npy")
  #X_Train = np.concatenate((data_x_p1,data_x_p2,data_x_p3,data_x_p4),axis = 0)

  #data_y_p1 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_y_Bacteria.npy")
  #data_y_p2 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_y_Normal.npy")
  data_y_p3 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_y_Virus.npy")
  #data_y_p4 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_y_Covid.npy")
  #y_Train = np.concatenate((data_y_p1,data_y_p2,data_y_p3,data_y_p4),axis = 0)

  #return X_Train, y_Train
  return data_x_p3[:,:,:,0], data_y_p3

In [4]:
#X,y = load_data()
#print(np.shape(X))

In [5]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 224  #@param  # 图像的高
        self.img_cols = 224  #@param  # 图像的宽
        self.channels =     1#@param  # 彩色图像
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100 #@param  #输入100维度输入
        self.init_name =   "Virus" #@param  # folder name
        # 设置学习率为0.0001
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    def build_generator(self):
        # 模型到深度及设置
        depths = [1024 , 512 , 256 , 128, 64, 32,3]  #@param
        model = Sequential()
        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim)) 
        model.add(Reshape((7, 7, 128))) 
        #（7，7，128）
        model.add(UpSampling2D())
        model.add(Conv2D(512, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        #（14，14，512）
        model.add(UpSampling2D()) 
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        #（28，28，256）
        model.add(UpSampling2D()) 
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        #（56，56，128）
        model.add(UpSampling2D()) 
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        #（112，112，64）
        model.add(UpSampling2D())        
        model.add(Conv2D(32, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
        #（224，224,3）
        #model.summary()
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        return Model(noise, img)
    def build_discriminator(self):
        Kernel_size = [4,3,3,3,3,3] #@param
        Stride = [2,2,2,2,2,2] #@param
        dropout = 0.17 #@param
        _momentum = 0.88 #@param
        _alpha = 0.1 #@param
        model = Sequential()
        #0
        model.add(Conv2D(32, kernel_size = Kernel_size[0], strides = Stride[0], input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=_alpha))
        model.add(Dropout(dropout))
        #1
        model.add(Conv2D(64, kernel_size= Kernel_size[1], strides = Stride[1], padding="same"))
        #model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=_momentum))
        model.add(LeakyReLU(alpha=_alpha))
        model.add(Dropout(dropout))
        #2
        model.add(Conv2D(128, kernel_size= Kernel_size[2], strides = Stride[2], padding="same"))
        model.add(BatchNormalization(momentum=_momentum))
        model.add(LeakyReLU(alpha=_alpha))
        model.add(Dropout(dropout))
        #3
        model.add(Conv2D(256, kernel_size=Kernel_size[3], strides= Stride[3], padding="same"))
        model.add(BatchNormalization(momentum=_momentum))
        model.add(LeakyReLU(alpha=_alpha))
        model.add(Dropout(dropout))
        #4
        model.add(Conv2D(512, kernel_size= Kernel_size[4], strides= Stride[4], padding="same"))
        model.add(BatchNormalization(momentum=_momentum))
        model.add(LeakyReLU(alpha=_alpha))
        model.add(Dropout(dropout))    
        #5
        model.add(Conv2D(1024, kernel_size= Kernel_size[5], strides= Stride[5], padding="same"))
        model.add(BatchNormalization(momentum=_momentum))
        model.add(LeakyReLU(alpha=_alpha))
        model.add(Dropout(dropout))                 
        #6
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        #model.summary()
        img = Input(shape=self.img_shape)
        validity = model(img)
        return Model(img, validity)
    def train(self, epochs, batch_size=128, save_interval=50):
        # Load the dataset
        X_train,y1 = load_data()
        #X_train = np.expand_dims(X_train, axis=3)
        index = np.shape(X_train)[1]
        pic_path = '/content/drive/MyDrive/UHCourse/ECE6397/mlresult_%d/%s_BS_'%(index,self.init_name)+str(batch_size)+'_SI'+str(save_interval)
        try:
          os.makedirs(pic_path)
        except:
          pass
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in notebook.tqdm(range(epochs)):
            # ---------------------
            #  Train Discriminator
            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)
            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            # ---------------------
            #  Train Generator
            # ---------------------
            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)
            # Plot the progress
            # If at save interval => save generated image samples
            if epoch % save_interval == 0:

                self.save_imgs(epoch,pic_path)
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
    def save_imgs(self, epoch,pic_path):
        r, c = 2, 2 #@param
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(pic_path+"/img_%d.png" % epoch)
        plt.close()

In [6]:
#dcgan = DCGAN()

In [ ]:
for Si in [50,100]:
    for Bs in [40,50,60,80,100,140,150]:
      dcgan = DCGAN() 
      dcgan.train(epochs=10000, batch_size=Bs, save_interval=Si)

0 [D loss: 3.000011, acc.: 31.25%] [G loss: 0.685126]
50 [D loss: 0.011809, acc.: 100.00%] [G loss: 0.721982]
100 [D loss: 0.000259, acc.: 100.00%] [G loss: 0.038561]
150 [D loss: 0.000259, acc.: 100.00%] [G loss: 0.066043]
200 [D loss: 0.000699, acc.: 100.00%] [G loss: 0.063610]
250 [D loss: 0.000077, acc.: 100.00%] [G loss: 0.037880]
300 [D loss: 0.000535, acc.: 100.00%] [G loss: 0.068522]
350 [D loss: 0.000324, acc.: 100.00%] [G loss: 0.014715]
400 [D loss: 0.000192, acc.: 100.00%] [G loss: 0.017582]
450 [D loss: 0.000098, acc.: 100.00%] [G loss: 0.016230]
500 [D loss: 0.561304, acc.: 80.00%] [G loss: 0.865898]
550 [D loss: 0.105368, acc.: 96.25%] [G loss: 0.855407]
600 [D loss: 0.011851, acc.: 100.00%] [G loss: 0.670622]
650 [D loss: 0.000697, acc.: 100.00%] [G loss: 0.227600]
700 [D loss: 0.000850, acc.: 100.00%] [G loss: 0.114088]
750 [D loss: 0.000414, acc.: 100.00%] [G loss: 0.049632]
800 [D loss: 0.000152, acc.: 100.00%] [G loss: 0.015881]
850 [D loss: 0.000247, acc.: 100.00%]